# Georeferenced maps in the State Library of Victoria's collections

This dashboard provides information on digitised maps from the [State Library of Victoria's collection](https://www.slv.vic.gov.au/search-discover/explore-collections-format/maps) that have been georeferenced using [Allmaps](https://allmaps.org). It is updated every two hours. Check [Allmaps Latest](https://latest.allmaps.org) for more recent additions.

See the [project documentation](https://wragge.github.io/slv-allmaps/) for information on how you can be involved!

In [ ]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from ipyleaflet import Map, Polygon
from ast import literal_eval
from urllib.parse import quote_plus
import arrow
from IPython.display import Markdown

In [ ]:
df = pd.read_csv("georeferenced_maps.csv", parse_dates=["modified", "created"])
df["geometry"] = df["geometry"].apply(literal_eval)

now = arrow.utcnow().to('Australia/Melbourne').format("h.MM a on D MMMM YYYY")

total_maps = df.shape[0]
total_manifests = df["ie_id"].nunique()

widgets.HTML(f"<p style='border: 1px solid; padding: 20px; font-size: 1.5rem;' >As of {now}, <b>{total_maps} maps</b> have been georeferenced from {total_manifests} collection items.</p>")

In [ ]:
Markdown("## Latest georeferenced maps")

You can explore the full list of georeferenced maps [using Datasette](https://glam-workbench.net/datasette-lite/?csv=https%3A%2F%2Fgithub.com%2Fwragge%2Fslv-allmaps%2Fblob%2Fmain%2Fgeoreferenced_maps_datasette.csv&install=datasette-homepage-table&install=datasette-json-html&fts=manifest_title%2Cmap_title) or the [Georeferenced maps explorer](https://slv.wraggelabs.com/geomaps/).

Here are the locations of the 20 maps most recently created or modified. Click on a highlighted area for more information.

In [ ]:
default_layout = widgets.Layout(width="100%", height="600px")
m = Map(
    center=(-37.815, 144.965),
    zoom=7,
    scroll_wheel_zoom=True,
    layout=default_layout,
)

for geomap in df.sort_values(["modified", "area"], ascending=False)[:20].itertuples():
    coords = [(c[1], c[0]) for c in geomap.geometry["coordinates"][0]]
    polygon = Polygon(
        locations=coords,
        color="blue",
        fill_color="blue",
        weight=1
    )
    html = ""
    html += f"<p style='line-height: 1.2;'><b>{geomap.manifest_title}</b></p><p style='line-height: 1.2;'>{geomap.map_title}</p>"
    html += f"<img src='{geomap.image_id}/full/!200,200/0/default.jpg'>"
    html += f"<p style='line-height: 1.2;'><a href='https://viewer.allmaps.org/?url={geomap.allmaps_map_id}'>View in the AllMaps viewer</a><br>"
    html += f"<a href='https://viewer.slv.vic.gov.au/?entity={geomap.ie_id}&file={geomap.fl_id}&mode=browse'>View in the SLV viewer</a></p>"
    polygon.popup = widgets.HTML(html)
    m.add(polygon)
m

Here are the details of the 20 maps most recently created or modified. 

In [ ]:
df["allmaps_link"] = df["allmaps_map_id"].apply(lambda x: f"<a href='https://viewer.allmaps.org/?url={x}'>View in the AllMaps viewer</a>")
df["slv_link"] = df.apply(lambda x: f"<a href='https://viewer.slv.vic.gov.au/?entity={x["ie_id"]}&file={x["fl_id"]}&mode=browse'>View in the SLV viewer</a>", axis=1)
df["thumbnail"] = df["image_id"].apply(lambda x: f"<img src='{x}/full/!100,100/0/default.jpg'>")

In [ ]:
df[["thumbnail", "manifest_title", "map_title", "modified", "allmaps_link", "slv_link"]].sort_values("modified", ascending=False)[:20].style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])]).hide()

In [ ]:
Markdown("## Georeferencing progress")

In [ ]:
alt.Chart(df).mark_bar().encode(
    x=alt.X("yearmonthdate(created):T").axis(format='%e %b %y').title(None),
    y=alt.Y("count():Q").title("number of maps")
).properties(width="container", title="maps georeferenced per day", padding=40)

In [ ]:
alt.Chart(df).transform_timeunit(
    date='yearmonthdate(created)'
).transform_window(
    frame=[None, 0],
    cumulative_count="count()",
    sort=[{"field": "date"}],
).mark_area().encode(
    x=alt.X("date:T").axis(format='%e %b %y').title(None),
    y=alt.Y("cumulative_count:Q").title("number of maps").stack(None)
).properties(width="container", title="total maps georeferenced over time", padding=40)